In [1]:
%env WANDB_PROJECT=aspect_explain_attack_vector_770m

env: WANDB_PROJECT=aspect_explain_attack_vector_770m


In [2]:
from project_dataset import load_dataset


In [4]:
from dataclasses import dataclass

@dataclass
class Args:
    model_name = "Salesforce/codet5p-770m"
    num_proc = 4
    batch_size = 2
    max_src_length = 1200
    max_des_length = 146
    data_cols = ["CVE ID", "explain", "func_before", "processed_func"]
    save_dir = 'tf_board'
    epochs = 11
    grad_acc_steps = 4
    lr = 5e-5
    log_freq = 10
    local_rank = -1
    deepspeed = None
    fp16 = False
    lr_warmup_steps = 200
    weight_decay = 0.05
    task = "attack_vector"
    prefix = "770m"
    
args = Args()

In [5]:
ds = load_dataset(args.task)

In [20]:
import os
from pathlib import Path
os.makedirs(args.save_dir, exist_ok=True)

In [7]:
from transformers import AutoTokenizer
codet5p_tokenizer = AutoTokenizer.from_pretrained(args.model_name)

In [8]:
import numpy as np

def preprocess_function(examples):
    source = [' '.join(ex) for ex in examples["func_before"]]
    target = [' '.join(ex) for ex in examples["explain"]]

    input_feature = codet5p_tokenizer(source, max_length=args.max_src_length, padding="max_length", truncation=True)
    labels = codet5p_tokenizer(target, max_length=args.max_des_length, padding="max_length", truncation=True)

    lables = labels["input_ids"].copy()
    # lables = np.where(lables != codet5p_tokenizer.pad_token_id, lables, -100)

    return {  "input_ids": input_feature["input_ids"],
              "attention_mask": input_feature["attention_mask"],
              "labels": lables}


tokenized_ds = ds.map(
  preprocess_function,
  remove_columns=args.data_cols,
  batched=True,
  num_proc=args.num_proc,
  batch_size=args.batch_size)

tokenized_ds

Map (num_proc=4):   0%|          | 0/4858 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/540 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/1350 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 4858
    })
    validation: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 540
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1350
    })
})

In [9]:
import torch
from transformers import AutoModelForSeq2SeqLM

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForSeq2SeqLM.from_pretrained(args.model_name).to(device)

In [10]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(
  codet5p_tokenizer,
  model=model,
  return_tensors="pt")

In [11]:
import evaluate
import numpy as np

rouge_metric = evaluate.load("rouge")

# define function for custom tokenization
def tokenize_sentence(arg):
  encoded_arg = codet5p_tokenizer(arg)
  return codet5p_tokenizer.convert_ids_to_tokens(encoded_arg.input_ids)

# define function to get ROUGE scores with custom tokenization
def metrics_func(eval_arg):
  preds, labels = eval_arg
  # Replace -100
  labels = np.where(labels != -100, labels, codet5p_tokenizer.pad_token_id)
  # Convert id tokens to text
  text_preds = codet5p_tokenizer.batch_decode(preds, skip_special_tokens=True)
  text_labels = codet5p_tokenizer.batch_decode(labels, skip_special_tokens=True)
  # Insert a line break (\n) in each sentence for ROUGE scoring
  text_preds = ["\n".join(p for p in text_preds)]
  text_labels = ["\n".join(l for l in text_labels)]
  # compute ROUGE score with custom tokenization
  return rouge_metric.compute(
    predictions=text_preds,
    references=text_labels,
    tokenizer=tokenize_sentence
  )

In [12]:
# for testing, show first 5 rows

from torch.utils.data import DataLoader

sample_dataloader = DataLoader(
  tokenized_ds["test"].with_format("torch"),
  collate_fn=data_collator,
  batch_size=5)

for batch in sample_dataloader:
  with torch.no_grad():
    preds = model.generate(
      batch["input_ids"].to(device),
      max_length=args.max_des_length,
    ).cpu()
  labels = batch["labels"].cpu()
  break

metrics_func([preds, labels])

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
Token indices sequence length is longer than the specified maximum sequence length for this model (726 > 512). Running this sequence through the model will result in indexing errors


{'rouge1': 0.3458980044345898,
 'rouge2': 0.10444444444444444,
 'rougeL': 0.20177383592017736,
 'rougeLsum': 0.31648351648351647}

In [13]:
preds.shape, labels.shape

(torch.Size([5, 146]), torch.Size([5, 146]))

In [14]:
from transformers import TrainingArguments

training_args = TrainingArguments(
        report_to='wandb',
        output_dir=args.save_dir,
        overwrite_output_dir=False,

        do_train=True,
        save_strategy='epoch',
        do_eval=True,

        num_train_epochs=args.epochs,
        per_device_train_batch_size=args.batch_size,
        gradient_accumulation_steps=args.grad_acc_steps,

        learning_rate=args.lr,
        weight_decay=args.weight_decay,
        warmup_steps=args.lr_warmup_steps,

        logging_dir=args.save_dir,
        logging_first_step=True,
        logging_steps=args.log_freq,
        save_total_limit=1,

        dataloader_drop_last=True,
        dataloader_num_workers=args.num_proc,

        local_rank=args.local_rank,
        deepspeed=args.deepspeed,
        fp16=args.fp16,
    )

In [ ]:
from transformers import Trainer
trainer = Trainer(
    model = model,
    args = training_args,
    data_collator = data_collator,
    compute_metrics = metrics_func,
    train_dataset = tokenized_ds["train"],
    eval_dataset = tokenized_ds["validation"],
    tokenizer = codet5p_tokenizer,
)

trainer.train()

/opt/conda/envs/vul-intext-reason/lib/python3.11/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Currently logged in as: dongchirua. Use `wandb login --relogin` to force relogin


/opt/conda/envs/vul-intext-reason/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
1,9.747500
10,9.009800
20,8.174800
30,7.076100
40,5.956900
50,5.287300
60,4.631800
70,4.250600
80,4.110100
90,3.972500


/opt/conda/envs/vul-intext-reason/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/opt/conda/envs/vul-intext-reason/lib/python3.11/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


In [17]:
trainer.save_model(f'{args.save_dir}/{args.prefix}/{args.task}')

## measure on test set

In [21]:
model.load_state_dict(torch.load(Path(f'{args.save_dir}/{args.prefix}/{args.task}')/"pytorch_model.bin"))

<All keys matched successfully>

In [22]:
sample_dataloader = DataLoader(
  tokenized_ds["test"].with_format("torch"),
  collate_fn=data_collator,
  batch_size=50,
  num_workers=args.num_proc)

rouge_list = []

for batch in sample_dataloader:
  with torch.no_grad():
    preds = model.generate(
      batch["input_ids"].to(device),
      max_length=args.max_des_length,
    ).cpu()
  labels = batch["labels"].cpu()

  rouge_result = metrics_func([preds, labels])
  print(rouge_result)
  rouge_list.append(rouge_result)

{'rouge1': 0.952480636382667, 'rouge2': 0.9001047120418848, 'rougeL': 0.7938036424534226, 'rougeLsum': 0.953025641025641}
{'rouge1': 0.9340181746345317, 'rouge2': 0.8268774703557311, 'rougeL': 0.6827340971947846, 'rougeLsum': 0.9313953488372093}
{'rouge1': 0.9546886358129392, 'rouge2': 0.88, 'rougeL': 0.7695662708989579, 'rougeLsum': 0.9543195266272191}
{'rouge1': 0.957010258915486, 'rouge2': 0.89247311827957, 'rougeL': 0.7821201758671226, 'rougeLsum': 0.9575381679389313}
{'rouge1': 0.9670567837017773, 'rouge2': 0.909366869037294, 'rougeL': 0.8153446033810143, 'rougeLsum': 0.966893039049236}
{'rouge1': 0.9700237222342032, 'rouge2': 0.9057173678532902, 'rougeL': 0.8164761699374595, 'rougeLsum': 0.9689545934530095}
{'rouge1': 0.9458172749943324, 'rouge2': 0.865502381492402, 'rougeL': 0.7689866243482204, 'rougeLsum': 0.9438899977822133}
{'rouge1': 0.9303331890956295, 'rouge2': 0.8398268398268398, 'rougeL': 0.7438338381652965, 'rougeLsum': 0.9271186440677968}
{'rouge1': 0.9564007421150278,

In [23]:
p = codet5p_tokenizer.batch_decode(preds[0], skip_special_tokens=True)
''.join(p).strip()

'v i a   a   c r a f t e d   d o c u m e n t  .'

In [24]:
l = codet5p_tokenizer.batch_decode(labels[0], skip_special_tokens=True)
''.join(l)

'v i a   a   c r a f t e d   d o c u m e n t  .'

In [25]:
rouge1_values = [d['rouge1'] for d in rouge_list]
rouge2_values = [d['rouge2'] for d in rouge_list]
rougeL_values = [d['rougeL'] for d in rouge_list]
rougeLsum_values = [d['rougeLsum'] for d in rouge_list]
rouge1_values = sum(rouge1_values) / len(rougeL_values)
average_rouge2 = sum(rouge2_values) / len(rougeL_values)
average_rougeL = sum(rougeL_values) / len(rougeL_values)
average_rougeLsum_values = sum(rougeLsum_values) / len(rougeL_values)

rouge1_values, average_rouge2, average_rougeL, average_rougeLsum_values

(0.9519546651010137,
 0.8773113429108883,
 0.7581029809854081,
 0.9508533866101629)